In [434]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
from tqdm import tqdm

## Web scrapping data source 
Checking justjoin.it main site and offers sites: html schema, where are most important information, etc.

In [413]:
#main site with criteria, to not download all offers
just_join_site = 'https://justjoin.it/remote_yes?keyword=analyst&keyword=data'

#no_fluff_site = "https://nofluffjobs.com/pl/artificial-intelligence?criteria=category%3Ddata,business-intelligence,business-analyst"

In [451]:
f = requests.get(just_join_site)
#print(f.text)

In [12]:
main_site = BeautifulSoup(f.text, "html.parser" )

In [452]:
#print(main_site)

In [18]:
main_site.title

<title>Job Offers IT - Just Join IT</title>

In [22]:
main_site.title.name

'title'

In [23]:
main_site.title.string

'Job Offers IT - Just Join IT'

In [25]:
main_site.title.parent.name

'head'

In [182]:
links_list = []
for link in main_site.find_all("a"):
    if "/offers/" in link.get("href"): 
        links_list.append(link.get("href"))
        
print(links_list)

['/offers/inter-cars-s-a-big-data-engineer', '/offers/softblue-software-qa-engineer', '/offers/crodu-supply-chain-audit-analyst-warszawa', '/offers/n-ix-data-architect-data-unit-krakow', '/offers/ciklum-senior-golang-developer-wroclaw-378779', '/offers/hcltech-data-analyst-warszawa', '/offers/miele-global-services-supply-chain-data-analyst-wroclaw', '/offers/ciklum-senior-ruby-on-rails-developer-gdansk', '/offers/miele-global-services-full-stack-supply-chain-analyst-wroclaw', '/offers/aura-data-analyst-krakow-379739', '/offers/the-i-factory-software-engineer-application-specialist-krakow', '/offers/aura-data-analyst-warszawa-379728', '/offers/emoti-group-web-analyst-poznan', '/offers/c-f-s-a-fullstack-net-developer-warszawa', '/offers/naviparking-business-analyst', '/offers/silky-coders-senior-ios-developer-gdansk-375955', '/offers/cognizant-data-engineer', '/offers/booksy-senior-business-analyst-warszawa', '/offers/nestle-polska-data-scientist-waraszawa', '/offers/ntt-data-business-so

In [31]:
for link in links_list[0:3]:
    site = "https://justjoin.it" + link
    print(site)

https://justjoin.it/offers/inter-cars-s-a-big-data-engineer
https://justjoin.it/offers/softblue-software-qa-engineer
https://justjoin.it/offers/crodu-supply-chain-audit-analyst-warszawa


In [453]:
offer = requests.get("https://justjoin.it/offers/softblue-software-qa-engineer")
#print(offer.text)

offer_soup = BeautifulSoup(offer.text, "html.parser" )


In [78]:
#position's name
offer_soup.find_all("h1")[1].text
niceToHaveSkills

'Software QA Engineer'

In [79]:
#retrieving attributes values
att_values = []
for i in offer_soup.find_all("div", {"class":"css-15wyzmd"}):
    att_values.append(i.text)

In [80]:
#retriving attributes names
att = []
for i in offer_soup.find_all("div", {"class":"css-qyml61"}):
    att.append(i.text)

In [61]:
#print(att, att_values)

In [145]:
offer_attributes = dict(zip(att, att_values))

offer_attributes

{'Type of work': 'Full-time',
 'Experience': 'Senior',
 'Employment Type': 'B2B',
 'Operating mode': 'Remote'}

In [412]:
salaries_soap = offer_soup.find_all("div",{"class":"css-j7qwjs"})
salaries_range = []
salaries_type = []
for i in salaries_soap:
    salaries_range.append(i.find("span",{"class":"css-1pavfqb"}).text)
    salaries_type.append(i.find("span",{"class":"css-1waow8k"}).text)

salary = dict(zip(salaries_range,salaries_type))

In [112]:
company_name = offer_soup.find_all("div", {"class": "css-mbkv7r"})[0].text
company_name

'SoftBlue'

In [113]:
offer_soup.find_all("h3")

[<h3 class="MuiTypography-root MuiTypography-h3 css-tdc3dz">Tech stack</h3>,
 <h3 class="MuiTypography-root MuiTypography-h3 css-1xhdodb">Job description</h3>]

In [140]:
tech_stack_soup = offer_soup.find("ul",{"class":"css-1uak81x"})
tech_stack = {}
for i in tech_stack_soup.find_all("div", {"class": "css-cjymd2"}):
    tech_stack[i.find("h6").text] = i.find("span").text
    
tech_stack

{'Automated Testing': 'advanced',
 'Robot Framework': 'advanced',
 'Python': 'regular',
 'SQL': 'regular',
 'CI/CD': 'regular',
 'Docker': 'regular',
 'Microsoft Azure Cloud': 'regular'}

In [212]:
## final solution
## writing all data to pandas
df = pd.DataFrame(columns = ["name", "link", "attributes", "salary", "company_name", "tech_stack", "description"])

for index, link in enumerate(links_list[0:5]):
    site = "https://justjoin.it" + link
    offer = requests.get(site)
    offer_soup = BeautifulSoup(offer.text, "html.parser")
    
    #job position name
    name = offer_soup.find_all("h1")[1].text
    
    #retriving attributes names
    att = []
    for i in offer_soup.find_all("div", {"class":"css-qyml61"}):
        att.append(i.text)
    
    #retrieving attributes values
    att_values = []
    for i in offer_soup.find_all("div", {"class":"css-15wyzmd"}):
        att_values.append(i.text)
        
    #attributtes: 'Type of work', Experience', 'Employment Type', 'Operating mode': 'Remote'}
    offer_attributes = dict(zip(att, att_values))
    
    #salary
    
    salaries_soap = offer_soup.find_all("div",{"class":"css-j7qwjs"})
    if salaries_soap:
        salaries_range = []
        salaries_type = []
        for i in salaries_soap:
            salaries_range.append(i.find("span",{"class":"css-1pavfqb"}).text)
            salaries_type.append(i.find("span",{"class":"css-1waow8k"}).text)

    salary = dict(zip(salaries_range,salaries_type))
    
    #company name
    company_name = offer_soup.find_all("div", {"class": "css-mbkv7r"})[0].text
    
    #technical stack
    tech_stack_soup = offer_soup.find("ul",{"class":"css-1uak81x"})
    tech_stack = {}
    for i in tech_stack_soup.find_all("div", {"class": "css-cjymd2"}):
        tech_stack[i.find("h6").text] = i.find("span").text
    #print({"name":name, "link":site, "attributes":offer_attributes, "salary": salary, "company_name":company_name, "tech_stack":tech_stack, "description":""})
    df.loc[index]={"name":name, "link":site, "attributes":offer_attributes, "salary": salary, "company_name":company_name, "tech_stack":tech_stack, "description":""}
    #df = df.reset_index(drop=True)

In [213]:
df.to_csv("offers_1.csv")
df

,name,link,attributes,salary,company_name,tech_stack,description
0,Big Data Engineer,https://justjoin.it/offers/inter-cars-s-a-big-...,"{'Type of work': 'Full-time', 'Experience': 'S...","{'15 000 - 17 500 PLN': 'Net/month - B2B', '12...",Inter Cars S.A.,"{'Scala': 'advanced', 'Hadoop': 'advanced', 'S...",
1,Software QA Engineer,https://justjoin.it/offers/softblue-software-q...,"{'Type of work': 'Full-time', 'Experience': 'S...",{'13 500 - 21 500 PLN': 'Net/month - B2B'},SoftBlue,"{'Automated Testing': 'advanced', 'Robot Frame...",
2,Supply Chain Audit Analyst,https://justjoin.it/offers/crodu-supply-chain-...,"{'Type of work': 'Full-time', 'Experience': 'M...",{'7 560 - 11 760 PLN': 'Net/month - B2B'},CRODU,"{'Communication Skills': 'advanced', 'Audit': ...",
3,Data Architect (Data Unit),https://justjoin.it/offers/n-ix-data-architect...,"{'Type of work': 'Full-time', 'Experience': 'S...","{'36 300 - 40 330 PLN': 'Net/month - B2B', '32...",N-iX,"{'AWS': 'master', 'Databases': 'master', 'Pyth...",
4,Senior Golang Developer,https://justjoin.it/offers/ciklum-senior-golan...,"{'Type of work': 'Full-time', 'Experience': 'S...",{'23 100 - 28 870 PLN': 'Net/month - B2B'},Ciklum,{'Go': 'advanced'},


In [456]:

data = offer_soup.find({id:"__NEXT_DATA__"})
#data

In [455]:
data.attrs

{}

In [256]:
json_data = json.loads(offer_soup.find("script", {"type": "application/json"}).text)

In [457]:
#json_data


In [274]:
for key in json_data:
    print(key)

props
page
query
buildId
runtimeConfig
isFallback
dynamicIds
gssp
appGip
scriptLoader


In [458]:
json_data['buildId']
json_data['runtimeConfig']['MAP_API_KEY']


'z7iGzSLh53F8TdrmZbkk'

In [277]:
props = json_data['props']
for key in props:
    print(key)

pageProps
__N_SSP


In [280]:
pageProps = props['pageProps']
for key in pageProps:
    print(key)

tenantConfig
unleashFlags
_sentryTraceData
_sentryBaggage
config
offer
isMobile
layoutMode
dehydratedState
showCurrencyDropdown


In [281]:
pageProps['offer']

{'slug': 'softblue-software-qa-engineer',
 'title': 'Software QA Engineer',
 'experienceLevel': {'label': 'Senior', 'value': 'senior'},
 'category': {'id': 5,
  'name': 'Python',
  'key': 'python',
  'parent_id': None,
  'lft': 9,
  'rgt': 10,
  'depth': 0,
  'children_count': 0,
  'created_at': '2023-09-25T22:41:46.648Z',
  'updated_at': '2023-09-25T22:41:46.648Z',
  'icon': {'d': 'M18.1966 7.24671C18.6897 7.24766 19.137 7.38458 19.5227 7.70035C19.8047 7.93129 20.0111 8.2207 20.1735 8.544C20.3412 8.87802 20.4431 9.23379 20.5248 9.59671C20.6132 9.98975 20.6901 10.3848 20.7327 10.7856C20.7687 11.1246 20.7813 11.4646 20.7657 11.8057C20.736 12.4553 20.6148 13.0901 20.4506 13.7167C20.2948 14.3118 19.9889 14.8172 19.5005 15.2013C19.1233 15.4979 18.694 15.6764 18.2183 15.7362C18.0461 15.7579 17.8713 15.7677 17.6976 15.7679L14.6462 15.7693L11.5104 15.7692V16.5245H15.7796V16.5816C15.7795 17.2179 15.7809 17.8543 15.7786 18.4906C15.7777 18.7571 15.7542 19.0219 15.6806 19.2798C15.5678 19.6749 15.

In [284]:
offer_data = json_data['props']['pageProps']['offer']
offer_data

{'slug': 'softblue-software-qa-engineer',
 'title': 'Software QA Engineer',
 'experienceLevel': {'label': 'Senior', 'value': 'senior'},
 'category': {'id': 5,
  'name': 'Python',
  'key': 'python',
  'parent_id': None,
  'lft': 9,
  'rgt': 10,
  'depth': 0,
  'children_count': 0,
  'created_at': '2023-09-25T22:41:46.648Z',
  'updated_at': '2023-09-25T22:41:46.648Z',
  'icon': {'d': 'M18.1966 7.24671C18.6897 7.24766 19.137 7.38458 19.5227 7.70035C19.8047 7.93129 20.0111 8.2207 20.1735 8.544C20.3412 8.87802 20.4431 9.23379 20.5248 9.59671C20.6132 9.98975 20.6901 10.3848 20.7327 10.7856C20.7687 11.1246 20.7813 11.4646 20.7657 11.8057C20.736 12.4553 20.6148 13.0901 20.4506 13.7167C20.2948 14.3118 19.9889 14.8172 19.5005 15.2013C19.1233 15.4979 18.694 15.6764 18.2183 15.7362C18.0461 15.7579 17.8713 15.7677 17.6976 15.7679L14.6462 15.7693L11.5104 15.7692V16.5245H15.7796V16.5816C15.7795 17.2179 15.7809 17.8543 15.7786 18.4906C15.7777 18.7571 15.7542 19.0219 15.6806 19.2798C15.5678 19.6749 15.

In [316]:
offerName = offer_data['title']
publishedAt = offer_data['publishedAt']
applyUrl = offer_data['applyUrl']
workingTime = offer_data['workingTime']['label']
cities = []
for i in offer_data['multilocation']:
    cities.append(i['city'])
    
company_name = offer_data['companyName']
workplaceType = offer_data['workplaceType']['label']

required_skills_json = offer_data['requiredSkills']

nice_to_have_skills_json = offer_data['niceToHaveSkills']

description = offer_data['body']
experienceLevel = offer_data['experienceLevel']['label']




In [315]:
required_skills = dict()
for skill in required_skills_json:
    required_skills[skill['name']] = skill['level']
print (required_skills)

{'Go': 4}


In [318]:
def skills_extraction(skills_json):
    skills = dict()
    for skill in skills_json:
        skills[skill['name']] = skill['level']
    return skills

In [321]:
skills_extraction(required_skills_json)

{'Go': 4}

In [320]:
skills_extraction(nice_to_have_skills_json)

{}

In [414]:
# What can be used:
#cities          +
#companyName     +
#workplaceType   +
#requiredSkills  +
#niceToHaveSkills+
#workingTime     +
#applyUrl        +
#publishedAt     +
#multilocation ?? - looks same as city

## Final solution
Can be extracted to .py file with imports (first cell) and function skills_extraction (above).

In [400]:
main_sites = ["https://justjoin.it/all-locations/analytics/remote_yes?orderBy=DESC&sortBy=newest",
              "https://justjoin.it/all-locations/data/experience-level_junior.mid/remote_yes?orderBy=DESC&sortBy=newest"]
#not one main site, just offers which intrests me

In [401]:
#provide just join it sites list with search criteria, return list of links to job offers
def get_offer_links(sites):
    
    links_set = set()
    for site in sites:
        f = requests.get(site)
        site_soup = BeautifulSoup(f.text, "html.parser")
        
        #using set because we need unique links
        for link in site_soup.find_all("a"):
            if "/offers/" in link.get("href"): 
                links_set.add(link.get("href"))
    
    return list(links_set)

In [402]:
links_list = get_offer_links(main_sites)


In [403]:
len(links_list)

121

In [447]:
## final version 2
## final solution
## writing all data to pandas


def get_offers_data_and_save(links_list):
    
    df = pd.DataFrame(columns = ["source_id","source", "name", "company_name","published_at", "attributes", "salary", "experience_level", "working_time", "location", "workplace_type", "required_skills", "nice_to_have_skills", "description","offer_url", "apply_url" ])

    for index, link in enumerate(tqdm(links_list)):
        site = "https://justjoin.it" + link
        offer = requests.get(site)
        offer_soup = BeautifulSoup(offer.text, "html.parser")

        #job position name
        name = offer_soup.find_all("h1")[1].text

        #retriving attributes names
        att = []
        for i in offer_soup.find_all("div", {"class":"css-qyml61"}):
            att.append(i.text)

        #retrieving attributes values
        att_values = []
        for i in offer_soup.find_all("div", {"class":"css-15wyzmd"}):
            att_values.append(i.text)

        #attributtes: 'Type of work', Experience', 'Employment Type', 'Operating mode'}
        offer_attributes = dict(zip(att, att_values))

        #salary

        salaries_soap = offer_soup.find_all("div",{"class":"css-j7qwjs"})
        if salaries_soap:
            salaries_range = []
            salaries_type = []
            for i in salaries_soap:
                salaries_range.append(i.find("span",{"class":"css-1pavfqb"}).text)
                salaries_type.append(i.find("span",{"class":"css-1waow8k"}).text)
        salary = dict(zip(salaries_range,salaries_type))

        #company name
        company_name = offer_soup.find_all("div", {"class": "css-mbkv7r"})[0].text

        #technical stack
        #tech_stack_soup = offer_soup.find("ul",{"class":"css-1uak81x"})
        #tech_stack = {}
        #for i in tech_stack_soup.find_all("div", {"class": "css-cjymd2"}):
        #    tech_stack[i.find("h6").text] = i.find("span").text


        json_data = json.loads(offer_soup.find("script", {"type": "application/json"}).text)
        offer_data = json_data['props']['pageProps']['offer']

        #offerName = offer_data['title']
        publishedAt = offer_data['publishedAt']
        applyUrl = offer_data['applyUrl']
        workingTime = offer_data['workingTime']['label']
        cities = []
        for i in offer_data['multilocation']:
            cities.append(i['city'])

        #company_name = offer_data['companyName']
        workplaceType = offer_data['workplaceType']['label']

        required_skills_json = offer_data['requiredSkills']

        nice_to_have_skills_json = offer_data['niceToHaveSkills']

        description = offer_data['body']
        experienceLevel = offer_data['experienceLevel']['label']

        #to change
        #ids = [json_data['buildId'],json_data['runtimeConfig']['MAP_API_KEY']]

        #writing to dataframe    
        #print({"name":name, "link":site, "attributes":offer_attributes, "salary": salary, "company_name":company_name, "tech_stack":tech_stack, "description":""})
        df.loc[index]={"source_id":None ,"source": "just_join_it",
                       "name":name, "company_name":company_name, "published_at":publishedAt, "attributes":offer_attributes, 
                       "salary":salary, "experience_level":experienceLevel, "working_time":workingTime, "location":cities,
                       "workplace_type":workplaceType, "required_skills":skills_extraction(required_skills_json), "nice_to_have_skills":skills_extraction(nice_to_have_skills_json), 
                       "description":description,"offer_url":site, "apply_url":applyUrl }   
    #df = df.reset_index(drop=True)

    print("Writing file...")
    today = time.strftime("%Y%m%d_%H%M")
    file_name = f"{today}_offers.csv"
    df.to_csv(file_name)
    print("Created", file_name)
    
    return df


In [445]:
df = get_offers_data_and_save(links_list[0:4])

100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.83it/s]

Writing file...
Created 20240429_1332_offers.csv


In [448]:
df

,source_id,source,name,company_name,published_at,attributes,salary,experience_level,working_time,location,workplace_type,required_skills,nice_to_have_skills,description,offer_url,apply_url
0,None,just_join_it,Data Engineer,MasterBorn Sp. z o.o.,2024-04-26T13:52:00.000Z,"{'Type of work': 'Full-time', 'Experience': 'M...",{'9 000 - 15 000 PLN': 'Net/month - B2B'},Mid,Full-time,"[Wrocław, Gdańsk, Kielce, Kraków, Bydgoszcz, W...",Remote,"{'SQL': 4, 'ETL': 4, 'English': 4, 'Polish': 3...",{},"<p><strong>Meet MasterBorn, a world-class soft...",https://justjoin.it/offers/masterborn-sp-z-o-o...,https://jobs.lever.co/masterborn-2/1c64e3ae-47...
1,None,just_join_it,Function Architecture Specialist,Reply Polska Sp. z o. o.,2024-04-26T15:00:04.861Z,"{'Type of work': 'Full-time', 'Experience': 'M...",{'9 000 - 16 000 PLN': 'Gross/month - Permanent'},Mid,Full-time,"[Łodź, Chorzów, Bielsko-Biała, Opole, Kielce, ...",Remote,"{'Enterprise Architect': 3, 'SYS-ML': 3}",{},<h2>Responsibilities</h2><p><br></p><ul>\n<li>...,https://justjoin.it/offers/reply-polska-sp-z-o...,https://www.reply.com/en/about/careers/pl/job-...
2,None,just_join_it,Data Engineer,Customertimes,2024-04-28T13:00:03.851Z,"{'Type of work': 'Full-time', 'Experience': 'M...",{'9 000 - 16 000 PLN': 'Gross/month - Permanent'},Mid,Full-time,[Warszawa],Remote,"{'Python': 5, 'AWS': 5, 'Glue': 4, 'Lambda': 4...",{},"<p>We are seeking an experienced, dedicated, a...",https://justjoin.it/offers/customertimes-data-...,None
3,None,just_join_it,Application Delivery Analyst,Simplicity Recruitment,2024-04-26T14:00:04.118Z,"{'Type of work': 'Full-time', 'Experience': 'M...",{'9 000 - 16 000 PLN': 'Gross/month - Permanent'},Mid,Full-time,"[Gdańsk, Kraków, Warszawa]",Remote,"{'English': 4, 'SQL': 2, 'WMS': 1}",{},<p><strong>🚀 Exciting Opportunity: Application...,https://justjoin.it/offers/simplicity-recruitm...,None


In [449]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 0 to 3
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   source_id            0 non-null      object
 1   source               4 non-null      object
 2   name                 4 non-null      object
 3   company_name         4 non-null      object
 4   published_at         4 non-null      object
 5   attributes           4 non-null      object
 6   salary               4 non-null      object
 7   experience_level     4 non-null      object
 8   working_time         4 non-null      object
 9   location             4 non-null      object
 10  workplace_type       4 non-null      object
 11  required_skills      4 non-null      object
 12  nice_to_have_skills  4 non-null      object
 13  description          4 non-null      object
 14  offer_url            4 non-null      object
 15  apply_url            2 non-null      object
dtypes: object(16)
mem

In [450]:
#TO DO
#adding data to file incrementally (in cycle, e.g. one time per day)
#but I can't see attribute to use, no ids, there is published_at, but looks weird (almost all dates set for today's morning)
#checking exisiting links doesn't seem to be efficient, moreover it looks that in future in the feature they can be not unique (they are uniqe in the moment, probably not in the timeline)